#Code written By: Sarthak Srivastava
Email: sarthak.s.1603@gmail.com

# Objective

To build a texutal analysis project. In order to run this notebook end to end you will need to take the following steps:

**All files required for running the code are present in the GitHub Repo**

1. Go to Runtime -> Run all

2. You will be asked for the permission to access Google Drive, you can give this permission as it is only for this runtime and next time it will be asked again, hence no safety concerns.

3. All the file paths are hardcoded in the code below.

4. Final output.csv file along with article files will be stored in the /content/ folder accessible from the left side.

5. In case any websites are not working, their URL will be shown at the end of execution.

# Installing Required Libraries

In [ ]:
import os
import glob
import pandas as pd
import string
!pip install requests
!pip install beautifulsoup4 --q
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data Pipeline Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

main_folder = '/content/drive/My Drive/20211030 Test Assignment/'

#Collecting all StopWords
folder_path = '/content/drive/My Drive/20211030 Test Assignment/StopWords/'
files = os.listdir(folder_path)
all_stop_words = []
for fil in files:
  file_path = os.path.join(folder_path, fil)
  stopwords = open(file_path, errors='ignore')
  stp = stopwords.read()
  print(file_path)
  all_stop_words.extend(stp.split())
all_stop_words = [word.lower() for word in all_stop_words]

#Collecting All positive and negative words
path = "/content/drive/My Drive/20211030 Test Assignment/MasterDictionary/"

positive_words = []
p = open(path+'positive-words.txt')
pw = p.read()

for word in pw.split():
  if word.lower() not in all_stop_words:
    positive_words.append(word)

negative_words = []
p = open(path+'negative-words.txt', errors='ignore')
pw = p.read()

for word in pw.split():
  if word.lower() not in all_stop_words:
    negative_words.append(word)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_Currencies.txt
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_DatesandNumbers.txt
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_Generic.txt
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_GenericLong.txt
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_Names.txt
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_Auditor.txt
/content/drive/My Drive/20211030 Test Assignment/StopWords/StopWords_Geographic.txt


# Text Extraction
In this part we will be downloading the .html file of the webpage using requests library and then parse it using beautiful soup.

In [ ]:
#Loading Input.xlsx file in pandas dataframe.
df = pd.read_excel(os.path.join(main_folder,'Input.xlsx'))

In [ ]:
#Creating a function that takes in URL and downloads the webpage.

def page_downloader(topics_url):
    response = requests.get(topics_url)
    if response.status_code==200:
        page_contents=response.text #saved the page in the variable.

    else:
        print("Cannot download Web-Page :( ")
        return None

    return(page_contents)

In [ ]:
#This function takes in BS4 object and returns page title and and its content in one string.
def web_scrapper(doc):

  page_title = ''
  page_content = ''

  #For Type1
  div_title = doc.find('div', class_='td-parallax-header')
  if div_title:
    header_tag = div_title.find('header')
    if header_tag:
      h1_tag = header_tag.find('h1')
      if h1_tag:
        page_title = h1_tag.text
    div_content = doc.find('div', class_='td-post-content tagdiv-type')
    if div_content:
      contents = div_content.contents
      for content in contents:
        page_content += content.text


  #For Type2
  div_title_1 = doc.find('div', class_='td_block_wrap tdb_title tdi_122 tdb-single-title td-pb-border-top td_block_template_1')
  if div_title_1:
     div_title_1 = div_title_1.find('div', class_='tdb-block-inner td-fix-index')
     contents = div_title_1.contents
     for content in contents:
      page_title += content.text

  div_content_1 = doc.find('div', class_='td_block_wrap tdb_single_content tdi_130 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')
  if div_content_1:
    div_content_1 = div_content_1.find('div', class_='tdb-block-inner td-fix-index')
    contents = div_content_1.contents
    for content in contents:
      page_content += content.text

  return page_title + page_content


#Text Processing

In this section we process the textand calculate the vaiours metrics.

## Tokenization and Puntuation removal

In [ ]:
#Starting with tokenization and Puntuation removal.

def tokenization(text):
  sentences = sent_tokenize(text)
  translation_table = str.maketrans('', '', string.punctuation)
  sentences = [sentence.translate(translation_table) for sentence in sentences]
  words = [word_tokenize(sentence) for sentence in sentences]
  return sentences,words


## Stopword Removal

In [ ]:
def remove_stopwords(words):
  words = [[word for word in sublist if word not in all_stop_words] for sublist in words]
  return words

In [ ]:
def total_words(words):
  total_words = sum(len(sublist) for sublist in words)
  return total_words

## Scores

In [ ]:
def scores(words):
  positive_score = 0
  negative_score = 0
  for sentence in words:
    for word in sentence:
      if word in positive_words:
        positive_score += 1
      if word in negative_words:
        negative_score += 1

  polarity_score = (positive_score - negative_score)/((positive_score + negative_score) + 0.000001)
  subjectivity_score = (positive_score + negative_score) / ((sum(len(sublist) for sublist in words)) + 0.000001)
  return positive_score, negative_score, polarity_score, subjectivity_score

## Analysis of Readability

In [ ]:
#Analysis of readabitily requires complex word count, which requires syllable count.

def count_syllables(word):

    total = 0
    complex_words = 0
    exceptions = ["es", "ed"]
    vowels = "aeiou"

    for sentence in words:
      for word in sentence:
        for exception in exceptions:
          if word.endswith(exception):
            word = word[:-len(exception)]
            vowel_count = sum(1 for char in word if char in vowels)
            if vowel_count > 2:
              complex_words += 1
            total += vowel_count



    return total, complex_words

In [ ]:
def aor(words):
  average_sentence_length = (sum(len(sublist) for sublist in words)) / len(words)
  cw = count_syllables(word)[1]
  tw = total_words(words)
  percentage_of_cw = cw/tw
  fog_index = 0.4 * (average_sentence_length + percentage_of_cw)

  return average_sentence_length, percentage_of_cw, fog_index



## Average Number of Words Per Sentence


In [ ]:
def anown(sum_of_words, sentences):
  avg = sum_of_words / len(sentences)
  return avg

## Complex Word Count


This is already counted before while creating count_syllable function, the result can be obtained by cwc = count_syllable(words)[1]

## Word Count

Word count can be calculated by total_words function previously created. Example: sum_of_words = total_words(words)


## Syllable Count Per Word


This is already calculated before under total syllables.

## Personal Pronouns

In [ ]:
def sum_personal_pronouns(text):
  #Use the very first extracted text as input without lowercasing to not count country US.
  pronoun_pattern = re.compile(r'\b(?:I|we|my|ours|us)\b', re.IGNORECASE)
  excluded_pattern = re.compile(r'\bUS\b', re.IGNORECASE)
  matches = pronoun_pattern.findall(text)
  # Excluding 'US' from the total count.
  excluded_matches = excluded_pattern.findall(text)
  total_count = len(matches) - len(excluded_matches)

  return total_count

## Average Word Length

In [ ]:
def average_word_length_is(words):
  char_count = 0
  for sentence in words:
    for word in sentence:
      char_count += len(word)
  sum_of_words = total_words(words)
  awl = char_count/sum_of_words
  return awl

# Final Output Generation

In [ ]:
#Loading Column names from the sample output file:
df_1 = pd.read_excel(os.path.join(main_folder,'Output Data Structure.xlsx'))
col_names = df_1.columns.to_list()

#Creating a blank output dataframe using the extracted column name.
output_df = pd.DataFrame(columns=col_names)

In [ ]:
#Checking blank dataframe
output_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH


In [ ]:
#Running a loop containg all the previously created functions.

error_on_webpages = []

for filename,url in zip(df['URL_ID'],df['URL']):

  #Downloading the page
  page = page_downloader(url)
  if page is None:
    error_on_webpages.append(url)
    print('Skipping')
    continue


  #Parsing the page
  doc = BeautifulSoup(page, 'html.parser')

  #Scrapping the page fpor content
  text_final = web_scrapper(doc)
  text_final = text_final.replace('\n', '')
  with open(('/content/'+filename+'.txt'), 'w') as file:
    file.write(text_final)
  print(f"{filename} is saved as .txt file")

  #Tokenization and Puntuation removal:
  sentences, words = tokenization(text_final)

  #Remove_Stopwords and count
  words = remove_stopwords(words)
  sum_of_words = total_words(words)


  #Calculate Scores
  positive_score, negative_score, polarity_score, subjectivity_score = scores(words)

  #AOR
  total_syllables , complex_words = count_syllables(words)
  average_sentence_length, percentage_of_cw, fog_index = aor(words)

  #Average number of Words per sentece
  anow_num = anown(sum_of_words,sentences)

  #Complex Word count same as complex_words
  #Word Count same as sum_of_words.
  #Syllable count per word same as total_syllables

  #Personal Pronouns count
  pp_count = sum_personal_pronouns(text_final)

  #Average word Length
  average_word_length = average_word_length_is(words)

  row_to_append = {'URL_ID' : filename,
                   'URL': url,
                   'POSITIVE SCORE': positive_score,
                   'NEGATIVE SCORE': negative_score,
                   'POLARITY SCORE': polarity_score ,
                   'SUBJECTIVITY SCORE': subjectivity_score,
                   'AVG SENTENCE LENGTH': average_sentence_length,
                   'PERCENTAGE OF COMPLEX WORDS':percentage_of_cw,
                   'FOG INDEX':fog_index,
                   'AVG NUMBER OF WORDS PER SENTENCE':anow_num,
                   'COMPLEX WORD COUNT': complex_words,
                   'WORD COUNT':sum_of_words,
                   'SYLLABLE PER WORD': total_syllables,
                   'PERSONAL PRONOUNS':pp_count,
                   'AVG WORD LENGTH': average_word_length}
  output_df = output_df.append(row_to_append,ignore_index=True)
  print(f"Successfully processed {filename}.")

output_df.to_csv('/content/output.csv', index=False)

print('')
print("All files processed, kindly find .txt article files and output.csv file in the '/content/' folder.")
print('The following webpages could not be downloaded:')
for item in error_on_webpages:
  print(item)

print('*******END OF EXECUTION*******')

blackassign0001 is saved as .txt file


<ipython-input-20-98410a77cef0>:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(row_to_append,ignore_index=True)


Successfully processed blackassign0001.
blackassign0002 is saved as .txt file


<ipython-input-20-98410a77cef0>:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(row_to_append,ignore_index=True)


Successfully processed blackassign0002.
blackassign0003 is saved as .txt file


...

<ipython-input-20-98410a77cef0>:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(row_to_append,ignore_index=True)


blackassign0099 is saved as .txt file
Successfully processed blackassign0099.


<ipython-input-20-98410a77cef0>:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(row_to_append,ignore_index=True)


blackassign0100 is saved as .txt file
Successfully processed blackassign0100.

All files processed, kindly find .txt article files and output.csv file in the '/content/' folder.
The following webpage could not be downloaded:
https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/
*******END OF EXECUTION*******


<ipython-input-20-98410a77cef0>:68: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(row_to_append,ignore_index=True)
